## Load data and define functions

In [ ]:
## Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import seaborn as sns
import os
import mne
import math
import pickle
import pprint
import re

## Importing functions
from mne import io, EvokedArray
# To make a pipeline
from sklearn.pipeline import make_pipeline
# Feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils import shuffle
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from mne.stats import permutation_t_test
# To classify
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from mne.decoding import (SlidingEstimator, cross_val_multiscore, LinearModel, get_coef, Scaler, Vectorizer)
from mne.decoding import GeneralizingEstimator

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from scipy import stats
from mne.stats import fdr_correction
from mne.stats import permutation_cluster_1samp_test

from pyriemann.estimation import ERPCovariances, XdawnCovariances, Xdawn
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, RidgeClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from sklearn import svm

In [ ]:
def getdata(epoch, t): #time window: 200 ms (SR=100Hz)
    data = epoch.get_data()
    data=data[:,:,t:t+20];
    return data

def getevents(epoch, n):
    events = epoch.events[:,2]
    events[events > 0] = n
    return events

In [ ]:
Eng = pd.read_excel("G:\내 드라이브\Research\Decoding\PsychopyViz\stimuli_Engv.xlsx")
Kor = pd.read_excel("G:\내 드라이브\Research\Decoding\PsychopyViz\stimuli_Korv.xlsx")

stim = []
for i in range(64):
    stim.append(Eng["word"][i])
    
for i in range(64):
    stim.append(Kor["word"][i])

In [ ]:
SUBJECTS=['R0781', 'R0809', 'R0810', 'R0811', 'R0812', 
         'R0813', 'R0814', 'R0818', 'R0828', 'R0831',
         'R0833', 'R0834', 'R0835', 'R0836'] #14

In [ ]:
EPOCHS = []
for subject in SUBJECTS:
    epoch = mne.read_epochs(f'CrossLingViz_EEG/noauto/{subject}_noauto_epo.fif')
    #epochs_ref = epochs_ref.filter(h_freq=100, l_freq = 1)
    EPOCHS.append(epoch)

### Plot

In [ ]:
ls_evoked1 = []
ls_evoked2 = []
ls_evoked3 = []
ls_evoked4 = []

for epochs_ref in EPOCHS:
    r1 = re.compile("Eng.*goat")
    r2 = re.compile("Eng.*duck")
    r3 = re.compile("Eng.*swan")
    r4 = re.compile("Eng.*lion")

    l1 = list(filter(r1.match, stim))
    l2 = list(filter(r2.match, stim))
    l3 = list(filter(r3.match, stim))
    l4 = list(filter(r4.match, stim))

    evoked1 = epochs_ref[l1].average()
    evoked2 = epochs_ref[l2].average()
    evoked3 = epochs_ref[l3].average()
    evoked4 = epochs_ref[l4].average()
    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)
    ls_evoked3.append(evoked3)
    ls_evoked4.append(evoked4)

In [ ]:
ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)
ga3 = mne.grand_average(ls_evoked3)
ga4 = mne.grand_average(ls_evoked4)

ga1.comment = 'English goat'
ga2.comment = 'English duck'
ga3.comment = 'English swan'
ga4.comment = 'English lion'

mne.viz.plot_compare_evokeds([ga1, ga2, ga3, ga4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
#200-400 KorV
Y.plot_topomap(
    times=[0, 1, 2, 3, 4],
    colorbar=False,
    size=1.5,
    time_format="Pattern %d"
);

In [ ]:
ls_evoked1 = []
ls_evoked2 = []
ls_evoked3 = []
ls_evoked4 = []

for epochs_ref in EPOCHS:
    r1 = re.compile("Kor.*goat")
    r2 = re.compile("Kor.*duck")
    r3 = re.compile("Kor.*swan")
    r4 = re.compile("Kor.*lion")

    l1 = list(filter(r1.match, stim))
    l2 = list(filter(r2.match, stim))
    l3 = list(filter(r3.match, stim))
    l4 = list(filter(r4.match, stim))

    evoked1 = epochs_ref[l1].average()
    evoked2 = epochs_ref[l2].average()
    evoked3 = epochs_ref[l3].average()
    evoked4 = epochs_ref[l4].average()
    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)
    ls_evoked3.append(evoked3)
    ls_evoked4.append(evoked4)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)
ga3 = mne.grand_average(ls_evoked3)
ga4 = mne.grand_average(ls_evoked4)

ga1.comment = 'Korean goat'
ga2.comment = 'Korean duck'
ga3.comment = 'Korean swan'
ga4.comment = 'Korean lion'

mne.viz.plot_compare_evokeds([ga1, ga2, ga3, ga4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []
ls_evoked3 = []
ls_evoked4 = []

for epochs_ref in EPOCHS:
    r1 = re.compile("Eng.*lean")
    r2 = re.compile("Eng.*cool")
    r3 = re.compile("Eng.*help")
    r4 = re.compile("Eng.*fill")

    l1 = list(filter(r1.match, stim))
    l2 = list(filter(r2.match, stim))
    l3 = list(filter(r3.match, stim))
    l4 = list(filter(r4.match, stim))

    evoked1 = epochs_ref[l1].average()
    evoked2 = epochs_ref[l2].average()
    evoked3 = epochs_ref[l3].average()
    evoked4 = epochs_ref[l4].average()
    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)
    ls_evoked3.append(evoked3)
    ls_evoked4.append(evoked4)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)
ga3 = mne.grand_average(ls_evoked3)
ga4 = mne.grand_average(ls_evoked4)

ga1.comment = 'English lean'
ga2.comment = 'English cool'
ga3.comment = 'English help'
ga4.comment = 'English fill'

mne.viz.plot_compare_evokeds([ga1, ga2, ga3, ga4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []
ls_evoked3 = []
ls_evoked4 = []

for epochs_ref in EPOCHS:
    r1 = re.compile("Kor.*lean")
    r2 = re.compile("Kor.*cool")
    r3 = re.compile("Kor.*help")
    r4 = re.compile("Kor.*fill")

    l1 = list(filter(r1.match, stim))
    l2 = list(filter(r2.match, stim))
    l3 = list(filter(r3.match, stim))
    l4 = list(filter(r4.match, stim))

    evoked1 = epochs_ref[l1].average()
    evoked2 = epochs_ref[l2].average()
    evoked3 = epochs_ref[l3].average()
    evoked4 = epochs_ref[l4].average()
    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)
    ls_evoked3.append(evoked3)
    ls_evoked4.append(evoked4)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)
ga3 = mne.grand_average(ls_evoked3)
ga4 = mne.grand_average(ls_evoked4)

ga1.comment = 'Korean lean'
ga2.comment = 'Korean cool'
ga3.comment = 'Korean help'
ga4.comment = 'Korean fill'

mne.viz.plot_compare_evokeds([ga1, ga2, ga3, ga4], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []

sg = ["Eng_lion1", "Eng_lion2", "Eng_lion3", "Eng_lion4", 
          "Eng_goat1", "Eng_goat2", "Eng_goat3", "Eng_goat4",
          "Eng_duck1", "Eng_duck2", "Eng_duck3", "Eng_duck4", 
          "Eng_swan1", "Eng_swan2", "Eng_swan3", "Eng_swan4"]
pl = ["Eng_lions1", "Eng_lions2", "Eng_lions3", "Eng_lions4", 
          "Eng_goats1", "Eng_goats2", "Eng_goats3", "Eng_goats4",
          "Eng_ducks1", "Eng_ducks2", "Eng_ducks3", "Eng_ducks4", 
          "Eng_swans1", "Eng_swans2", "Eng_swans3", "Eng_swans4"]

for epochs_ref in EPOCHS:

    evoked1 = epochs_ref[sg].average()
    evoked2 = epochs_ref[pl].average()

    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)

ga1.comment = 'English singular'
ga2.comment = 'English plural'

mne.viz.plot_compare_evokeds([ga1, ga2], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []

sg = ["Kor_lion1", "Kor_lion2", "Kor_lion3", "Kor_lion4", 
          "Kor_goat1", "Kor_goat2", "Kor_goat3", "Kor_goat4",
          "Kor_duck1", "Kor_duck2", "Kor_duck3", "Kor_duck4", 
          "Kor_swan1", "Kor_swan2", "Kor_swan3", "Kor_swan4"]
pl = ["Kor_lions1", "Kor_lions2", "Kor_lions3", "Kor_lions4", 
          "Kor_goats1", "Kor_goats2", "Kor_goats3", "Kor_goats4",
          "Kor_ducks1", "Kor_ducks2", "Kor_ducks3", "Kor_ducks4", 
          "Kor_swans1", "Kor_swans2", "Kor_swans3", "Kor_swans4"]

for epochs_ref in EPOCHS:

    evoked1 = epochs_ref[sg].average()
    evoked2 = epochs_ref[pl].average()

    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)

ga1.comment = 'Korean singular'
ga2.comment = 'Korean plural'

mne.viz.plot_compare_evokeds([ga1, ga2], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []

pres = ["Eng_cools1","Eng_cools2", "Eng_cools3", "Eng_cools4", 
          "Eng_helps1", "Eng_helps2", "Eng_helps3", "Eng_helps4",
          "Eng_leans1", "Eng_leans2", "Eng_leans3", "Eng_leans4",
          "Eng_fills1", "Eng_fills2", "Eng_fills3", "Eng_fills4"]
past = ["Eng_cooled1","Eng_cooled2", "Eng_cooled3", "Eng_cooled4", 
          "Eng_helped1", "Eng_helped2", "Eng_helped3", "Eng_helped4",
          "Eng_leaned1", "Eng_leaned2", "Eng_leaned3", "Eng_leaned4",
          "Eng_filled1", "Eng_filled2", "Eng_filled3", "Eng_filled4"]

for epochs_ref in EPOCHS:

    evoked1 = epochs_ref[pres].average()
    evoked2 = epochs_ref[past].average()

    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)

ga1.comment = 'English present'
ga2.comment = 'English past'

mne.viz.plot_compare_evokeds([ga1, ga2], picks='Cz', ylim=dict(eeg=[-10, 10]));

In [ ]:
ls_evoked1 = []
ls_evoked2 = []

pres = ["Kor_cools1","Kor_cools2", "Kor_cools3", "Kor_cools4", 
          "Kor_helps1", "Kor_helps2", "Kor_helps3", "Kor_helps4",
          "Kor_leans1", "Kor_leans2", "Kor_leans3", "Kor_leans4",
          "Kor_fills1", "Kor_fills2", "Kor_fills3", "Kor_fills4"]
past = ["Kor_cooled1","Kor_cooled2", "Kor_cooled3", "Kor_cooled4", 
          "Kor_helped1", "Kor_helped2", "Kor_helped3", "Kor_helped4",
          "Kor_leaned1", "Kor_leaned2", "Kor_leaned3", "Kor_leaned4",
          "Kor_filled1", "Kor_filled2", "Kor_filled3", "Kor_filled4"]

for epochs_ref in EPOCHS:

    evoked1 = epochs_ref[pres].average()
    evoked2 = epochs_ref[past].average()

    ls_evoked1.append(evoked1)
    ls_evoked2.append(evoked2)

ga1 = mne.grand_average(ls_evoked1)
ga2 = mne.grand_average(ls_evoked2)

ga1.comment = 'Korean present'
ga2.comment = 'Korean past'

mne.viz.plot_compare_evokeds([ga1, ga2], picks='Cz', ylim=dict(eeg=[-10, 10]));

## Perform temporal decoding

### NOUNS, VERBS

In [ ]:
#200 ms window

a = "Eng.*duck"
b = "Eng.*goat"
c = "Eng.*swan"
d = "Eng.*lion"

a1 = "Kor.*duck"
b1 = "Kor.*goat"
c1 = "Kor.*swan"
d1 = "Kor.*lion"

e = "Eng.*fill"
f = "Eng.*cool"
g = "Eng.*help"
h = "Eng.*lean"

e1 = "Kor.*fill"
f1 = "Kor.*cool"
g1 = "Kor.*help"
h1 = "Kor.*lean"

r1 = re.compile(e)
r2 = re.compile(f)
r3 = re.compile(g)
r4 = re.compile(h)
l1 = list(filter(r1.match, stim))
l2 = list(filter(r2.match, stim))
l3 = list(filter(r3.match, stim))
l4 = list(filter(r4.match, stim))

listOftuples=[]

for subject in SUBJECTS:
    accuracy = []
    epoch = mne.read_epochs(f"CrossLingViz_EEG/noauto/{subject}_noauto_epo.fif")
    epoch = epoch.copy().resample(100)
     
    train1 = epoch[l1]
    train2 = epoch[l2]
    train3 = epoch[l3]
    train4 = epoch[l4]

    # Prepare train & test data
    t = 0 #50 (100 Hz: 10 = -200ms, 30 = 0ms)
    while t < 130:

        trainx = np.concatenate((getdata(train1, t),getdata(train2, t), getdata(train3, t), getdata(train4, t)), axis=0)
        trainy = np.concatenate((getevents(train1, 1),getevents(train2, 2), getevents(train3, 3), getevents(train4, 4)), axis=0)

        train_data, train_labels = shuffle(trainx, trainy, random_state=4)

        clf = make_pipeline(Xdawn(), Vectorizer(),StandardScaler(), LDA(shrinkage='auto', solver='eigen'))
        scores = cross_val_score(clf, train_data, train_labels, cv=5, scoring='accuracy')

        accuracy.append(scores.mean())
        t += 10
        
    times = list(np.arange(-300,1000,100))
    for i in range(len(times)):
        listOftuples.append(tuple([subject, times[i], accuracy[i]]))    

    df = pd.DataFrame(listOftuples, columns = ['subject', 'time', 'score']) 

#df.to_pickle('Temporal decoding/KVerball_lda_200ms_stand.pkl')

In [ ]:
#plot

plt.figure() 
ax = sns.lineplot(x = 'time', y = 'score', data = df, color='black',label='Eng nouns')
ax.axhline(.25, color='k', linestyle='--')
ax.set_xlabel('Time (s)')
ax.set_ylabel('AUC')  # Area Under the Curve
ax.axvline(.0, color='k', linestyle='dotted')
#ax.set_xlim([-.2,.8])
#ax.set_ylim([0.2,.35])
ax.legend()

In [ ]:
#permutation

df1 = df
TimesX = np.unique(df1.time)
ENG = pd.pivot_table(df1, values = 'score', columns = 'time', index = 'subject', 
                           aggfunc={'score': np.mean}).values

T_obs, clusters, cluster_p_values, H0 = \
    permutation_cluster_1samp_test(ENG - 0.25, n_permutations=10000)
